In [0]:
#This Code predict Assignee for Roslyn Issues with Title

import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
from sklearn import preprocessing
import keras
import numpy as np

url = "https://tfhub.dev/google/universal-sentence-encoder-large/3"
embed = hub.Module(url)

data = pd.read_csv('IssueDetailsRoslyn_20102019.csv', encoding='latin-1')

y = list(data['Assignee']) 
x = list(data['Title']) 

le = preprocessing.LabelEncoder()
le.fit(y)

categories = list(set(y))

def encode(le, labels):
    enc = le.transform(labels)
    return keras.utils.to_categorical(enc)

def decode(le, one_hot):
    dec = np.argmax(one_hot, axis=1)
    return le.inverse_transform(dec)

test = encode(le,categories)

untest = decode(le, test)

x_enc = x
y_enc = encode(le, y)

#80% / 20% train / test split:

train_size = int(len(x) * .8)

x_train = np.asarray(x_enc[:train_size])
y_train = np.asarray(y_enc[:train_size])

x_test = np.asarray(x_enc[train_size:])
y_test = np.asarray(y_enc[train_size:])

In [0]:
from keras.layers import Input, Lambda, Dense
from keras.models import Model
import keras.backend as K

def UniversalEmbedding(x):
    return embed(tf.squeeze(tf.cast(x, tf.string)))

In [14]:
input_text = Input(shape=(1,), dtype=tf.string)
embedding = Lambda(UniversalEmbedding, output_shape=(512, ))(input_text)
dense = Dense(256, activation='relu')(embedding)
pred = Dense(len(categories), activation='softmax')(dense)
model = Model(inputs=[input_text], outputs=pred)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [15]:
with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())  
    session.run(tf.tables_initializer())
    history = model.fit(x_train, y_train, epochs=30, batch_size=32)
    model.save_weights('./model_roslyn_Title_Assignee_Prediction.h5')

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/30



716/716 [==============================] - 18s 25ms/step - loss: 5.5351 - acc: 0.0656
Epoch 2/30
716/716 [==============================] - 9s 12ms/step - loss: 4.7703 - acc: 0.0922
Epoch 3/30
716/716 [==============================] - 8s 11ms/step - loss: 4.3125 - acc: 0.1494
Epoch 4/30
716/716 [==============================] - 8s 11ms/step - loss: 4.1056 - acc: 0.1704
Epoch 5/30
716/716 [==============================] - 8s 11ms/step - loss: 3.9389 - acc: 0.2025
Epoch 6/30
716/716 [==============================] - 8s 11ms/step - loss: 3.8000 - acc: 0.2067
Epoch 7/30
716/716 [==============================] - 8s 11ms/step - loss: 3.6665 - acc: 0.2249
Epoch 8/30
716/716 [==============================] - 8s 11ms/step - loss: 3.5406 - acc: 0.2221
Epoch 9/30
716/716 [==============================] - 8s 11ms/step - loss: 3.4219 - acc: 0.2444
Epoch 10/30
716/716 [==============================] - 8s 12ms/step - loss: 3.3144 - acc: 0.2374
Epoch 11/30
716/716 [============================

In [0]:
with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())
    session.run(tf.tables_initializer())
    model.load_weights('./model_roslyn_Title_Assignee_Prediction.h5')  
    predicts = model.predict(x_test, batch_size=32)  

In [17]:
y_test = np.asarray(y_enc[train_size:])


y_test = decode(le, y_test)
y_preds = decode(le, predicts)

from sklearn import metrics

metrics.confusion_matrix(y_test, y_preds)

print(metrics.classification_report(y_test, y_preds))


                                                                                                                      precision    recall  f1-score   support

                                                                                                      333fred|gafter       0.00      0.00      0.00         1
                                                                            333fred|jaredpar|heejaechang|tmat|agocke       0.00      0.00      0.00         1
   333fred|jasonmalinowski|ryzngard|jaredpar|agocke|gafter|RikkiGibson|VSadov|AlekseyTs|cston|jcouv|chsienki|fayrose       0.00      0.00      0.00         1
                                                                                               333fred|tmat|jaredpar       0.00      0.00      0.00         1
                                                                                                           AlekseyTs       0.00      0.00      0.00         3
                                                   

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [7]:
#statistics
data['word_count'] = data['Description'].apply(lambda x: len(str(x).split()))
desc_lengths = list(data['word_count'])
print("Number of descriptions:",len(desc_lengths),
      "\nAverage word count", np.average(desc_lengths),
      "\nMinimum word count", min(desc_lengths),
      "\nMaximum word count", max(desc_lengths))

Number of descriptions: 895 
Average word count 90.37430167597765 
Minimum word count 1 
Maximum word count 1220


# New Section